In [1]:
import time 
import urllib.request as req
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from PIL import Image
import requests
import os
import numpy as np

In [6]:
webdriver_path = 'C:/Users/FRONTIS/Desktop/윤서진 문서/chromedriver_win32/chromedriver.exe'
site_url = "https://www.google.com/?gws_rd=ssl" #GOOGLE
dir_path = 'C:/Users/FRONTIS/Desktop/윤서진 문서/Crawling/skin care wrinkle B&F'

search_keyword = 'skin care wrinkle before and after'

need_image_count = 100

show_more_btn_xpath = '//*[@id="islmp"]/div/div/div/div/div[5]/input'
search_end_label_xpath = '/html/body/div[2]/c-wiz/div[3]/div[1]/div/div/div/div/div[4]/div[2]/div[1]/div'

In [11]:
# webdriver 열기
def open_webdriver(webdriver_path, site_url):
    driver = webdriver.Chrome(webdriver_path)
    driver.get(site_url)
    return driver

def is_can_pageDown(driver):
    show_more_btn = driver.find_element_by_xpath(show_more_btn_xpath)
    img_end_webelement = driver.find_element_by_xpath(search_end_label_xpath)
    
    return not show_more_btn.is_displayed() or img_end_webelement.is_displayed()

def is_show_reloadBtn(driver):
    show_more_btn = driver.find_element_by_xpath(show_more_btn_xpath)
    return show_more_btn.is_displayed()
    
def is_end_search(driver):
    img_end_webelement = driver.find_element_by_xpath(search_end_label_xpath)
    
    return img_end_webelement.is_displayed()

def pageDown(driver):
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    print('pageDown')
    
# 구글 이미지 검색
def google_image_search(driver, keyword):
    #검색어 창 선택
    e = driver.find_element_by_xpath("//*[@id='tsf']/div[2]/div[1]/div[1]/div/div[2]/input")

    #검색어 입력
    e.send_keys(keyword)

    #엔터 입력
    e.submit()

    # 결과가 나올때까지 대기
    time.sleep(1)
    
    # 이미지 탭으로 이동
    driver.find_element_by_xpath('//*[@id="hdtb-msb-vis"]/div[2]/a').click()
    
# 이미지 리소스를 찾음
def find_imgsrc_classInHTML(html, class_name):
    soup = BeautifulSoup(html,'html.parser')
    img_sources = soup.findAll('img', class_ = class_name) 
    return img_sources

# 여러 이미지 리소스 중 http 리소스를 찾음
def find_http_imgSrc(img_sources):
    http_source = ''
    for img_source in img_sources:
        if img_source['src'].startswith('http'):
            http_source = img_source['src']
    return http_source

# 디버깅용도: html 을 텍스트로 입력받아 파일로 출력
def write_html(webdriver, dir_path, file_name):
    make_folder_not_exist(dir_path)
    
    with open(dir_path + '/' + file_name, 'w', encoding = 'utf-8') as f:
        f.write(webdriver.page_source)

# 이미지 다운로드
def img_download(dir_path, image_name, img_src):
    if not os.path.exists(dir_path):
        raise Exception('Please Make directory: ', dir_path)
    
    response = requests.get(url = img_src)
    full_image_path = dir_path + '/' + image_name
    
    if response.status_code == 200:
        with open(dir_path + '/' + image_name, 'wb') as f:
            f.write(response.content)
            print('save complete: ', full_image_path)
    else:
        print('save failed url: ', img_src, ' status code: ', response.status_code)

# 폴더 생성
def make_folder_not_exist(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print('Make folder: ', folder_path)

In [21]:
def main():
    driver = open_webdriver(webdriver_path = webdriver_path, site_url = site_url)
    time.sleep(1)

    google_image_search(driver = driver, keyword = search_keyword)
    time.sleep(0.5)

    # 폴더 생성
    make_folder_not_exist(dir_path)

    # data 전체를 포괄하는 Web-Element 컨테이너
    data_element_container = driver.find_element_by_xpath('//*[@id="islrg"]/div[1]') # data element가 모두 모여있는 요소

    # 이미지 로딩
    while(True):
        # 데이터 Web-Element 각각을 가지고 있는 리스트
        data_element_List = data_element_container.find_elements_by_class_name('isv-r.PNCib.MSM1fd.BUooTd')
        if (len(data_element_List) >= need_image_count):
            print("Load Image Enough!")
            break
        else:
            pageDown(driver = driver)
            time.sleep(1)
            if is_end_search(driver = driver):
                print("Search End!")
                break
            elif is_show_reloadBtn(driver = driver):
                show_more_btn = driver.find_element_by_xpath(show_more_btn_xpath)
                show_more_btn.click()

    element_index = 0
    while(True):
        data_element = data_element_List[element_index]

        if (element_index == need_image_count):
            # 내가 원하는 개수만큼 다운로드
            break
        else:
            # 다운 받으려는 Element를 클릭해야 다운로드 URL을 얻을수 있음
            # 여러개의 클래스중 http URL이 존재하는 클래스 선택
            # 해당 URL로 이미지 다운로드

            data_element_List[element_index].click()

            # Image URL이 얻어질때까지 루프를 돔. (한번만 url을 가져오면 잘 안됨. 구글이 막아놓은것 같음)
            img_url = ''
            no_image_source_log_count = 0
            skip = False
            
            while(True):
                # Soup
                img_sources = find_imgsrc_classInHTML(driver.page_source, class_name = 'n3VNCb')
                img_url = find_http_imgSrc(img_sources)
                
                if img_url == '':
                    no_image_source_log_count += 1
                    
                    if no_image_source_log_count < 10:
                        print('No image source!, sleep case 1, Element index: ', element_index)
                        time.sleep( np.random.uniform(low = 0, high = 1, size = 1) )
                    elif no_image_source_log_count < 20:
                        print('No image source!, sleep case 2, Element index: ', element_index)
                        time.sleep( np.random.uniform(low = 1, high = 2, size = 1) )
                    else:
                        print('No image source!, skip, Element index: ', element_index)
                        skip = True
                        break
                else:
                    break
            
            if skip == False:
                image_name = search_keyword + '_' + str(element_index) + '.png'
                img_download(dir_path = dir_path, image_name = image_name, img_src = img_url) 

        element_index += 1

In [22]:
main()

pageDown
Load Image Enough!
1
No image source!, sleep case 1, Element index:  0
2
No image source!, sleep case 1, Element index:  0
save complete:  C:/Users/FRONTIS/Desktop/윤서진 문서/Crawling/skin care wrinkle before and after_0.png
save complete:  C:/Users/FRONTIS/Desktop/윤서진 문서/Crawling/skin care wrinkle before and after_1.png
1
No image source!, sleep case 1, Element index:  2
2
No image source!, sleep case 1, Element index:  2
3
No image source!, sleep case 1, Element index:  2
save complete:  C:/Users/FRONTIS/Desktop/윤서진 문서/Crawling/skin care wrinkle before and after_2.png
save complete:  C:/Users/FRONTIS/Desktop/윤서진 문서/Crawling/skin care wrinkle before and after_3.png
save complete:  C:/Users/FRONTIS/Desktop/윤서진 문서/Crawling/skin care wrinkle before and after_4.png
1
No image source!, sleep case 1, Element index:  5
2
No image source!, sleep case 1, Element index:  5
3
No image source!, sleep case 1, Element index:  5
4
No image source!, sleep case 1, Element index:  5
5
No image sou